In [ ]:
import pickle
import sys
sys.path.append('/nfs/gns/literature/Santosh_Tirunagari/test Gitlab/epmc-ml-misc-library/')
import capo_tools_lib
import glob
import json

import random
from spacy.util import minibatch, compounding

json_path = '/nfs/gns/literature/machine-learning/cleaned_dataset/'
trainPMCids, devPMCids, testPMCids = capo_tools_lib.create_train_dev_test_splits(json_path, percentage = 0.70)

   
    
def convert_json_to_spacy_format(json_path):

    training_data = []
    all_files = sorted(glob.glob(json_path + '*.json*'))
    for each_manually_annotated_json in all_files:
        with open(each_manually_annotated_json) as json_file_ner_rel:
            json_data = json.loads(json_file_ner_rel.read())
            
            for articles in json_data:
                pmc_id = articles  # json_data[articles]
                if pmc_id in trainPMCids+devPMCids:
                    for each_annotation in json_data[articles]['annotations']:
                        text = each_annotation['sent'].encode('utf-8').decode('utf-8')
                        entities = []
                        ner = each_annotation['ner']
                        if ner:
                            for each_ner in ner:
                                point_start = each_ner[0]
                                point_end = each_ner[1]
                                label = each_ner[3]
                                entities.append((point_start, point_end,label))

                            training_data.append((text, {"entities" : entities}))

#                 print(training_data)

#     with open(output_file, 'wb') as fp:
#         pickle.dump(training_data, fp)
    return training_data




In [ ]:
train_data = convert_json_to_spacy_format(json_path)

In [3]:
###### Spacy pretraining language model
import glob
import jsonlines 
from tqdm import tqdm

textfiles_path = '/nfs/misc/literature/yangx/resources/xmls/articles/fulltext_token/'
text_files = sorted(glob.glob(textfiles_path + '*.xml*'))
result_dump = '/nfs/gns/literature/Santosh_Tirunagari/pretrained_word_embeddings/Data/'


with jsonlines.open(result_dump+'europepmc.jsonl', mode='w') as writer:
    for each_file in tqdm(text_files):
        with open(each_file) as xml_file:
            xml_text = xml_file.read()
            for text in xml_text.split('\n'):
                writer.write({'text':text})

100%|██████████| 100/100 [01:53<00:00,  1.13s/it]


In [5]:
###### Spacy pretraining language model
import glob
import jsonlines 
from tqdm import tqdm
import pandas as pd


train_set = '/nfs/gns/literature/machine-learning/evaluation/300articles/CSV formats/train.csv'
dev_set = '/nfs/gns/literature/machine-learning/evaluation/300articles/CSV formats/dev.csv'
result_dump = '/nfs/gns/literature/Santosh_Tirunagari/pretrained_word_embeddings/Data/'

train_df = pd.read_csv(train_set, sep = '\t', names = ['pmcid', 'sentence','ner'])
dev_df = pd.read_csv(dev_set, sep = '\t', names = ['pmcid', 'sentence','ner'])

text_train_dev = []
text_train_dev.extend(train_df['sentence'].values)
text_train_dev.extend(dev_df['sentence'].values)
text_train_dev

with jsonlines.open(result_dump+'golddata_train_dev.jsonl', mode='w') as writer:
    for text in text_train_dev:
        if(len(text)>50):
            writer.write({'text':text.strip()})

In [1]:
# !python -m spacy pretrain Data/golddata_train_dev.jsonl /nfs/gns/literature/Santosh_Tirunagari/pretrained_word_embeddings/models/pubmed-pmc models/gold_pretrained --use-vectors


In [ ]:
import scispacy
import spacy
# Setting up the pipeline and entity recognizer.
# model = 'en_core_sci_md'
model = 'en_core_sci_md'

if model is not None:
    nlp = spacy.load(model)  # load existing spacy model
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  # create blank Language class
    print("Created blank 'en' model")

    
# nlp = spacy.blank("en")
# textcat = nlp.create_pipe("textcat", config={"architecture": "simple_cnn", "exclusive_classes": True}))
# textcat.add_label("LABEL1")
# textcat.add_label("LABEL2")
# # Alternatively, instead of adding all your labels explicitly, you could pass all your examples
# # into textcat.begin_training, like this: textcat.begin_training(get_gold_tuples=lambda: my_data)
# # It's fine to add the labels and not pass in the data, though. The nlp.begin_training() method will
# # work the same as well, if you have other components in your pipeline you want to train.
# optimizer = textcat.begin_training()
# # Now that we have our model, we can load in the pretrained weights.
# with open(path_to_pretrained_weights, "rb") as file_:
#     textcat.model.tok2vec.from_bytes(file_.read())
# # Now we can proceed with training
# for epoch in range(nr_epoch):
#     random.shuffle(train_data)
#     for batch in minibatch(train_data, size=batch_size):
#         X, y = zip(*batch)
#         textcat.update(X, y, sgd=optimizer) 

In [ ]:
if model is None:
    optimizer = nlp.begin_training()
else:
    optimizer = nlp.entity.create_optimizer()   
    
optimizer    

In [ ]:
from datetime import datetime

    
    
def train_spacy(data,nlp,iterations,optimizer):
    TRAIN_DATA = data
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner') # nlp.add_pipe(ner, last=True)
        nlp.add_pipe(ner)
        print('new pipeline added')
    else:
        print('model\'s ner added')
        ner = nlp.get_pipe('ner')
    
    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])
    print('labels have been added')
    
    # Get names of other pipes to disable them during training to train only NER
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    print('Training Started')
    with nlp.disable_pipes(*other_pipes):  # only train NER
        for itn in range(iterations):
            random.shuffle(TRAIN_DATA)
            losses = {}
            batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35,
                           losses=losses)

            now = datetime.now()
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
            print("date and time =", dt_string)
            print('Losses', losses)
            print('iteration-- '+str(itn))
            print('\n')

        
    return nlp



In [ ]:
nlp = spacy.blank("en")
ner = nlp.create_pipe("ner", config={"architecture": "simple_cnn", "exclusive_classes": True}))
path_to_pretrained_weights = '/nfs/gns/literature/Santosh_Tirunagari/pretrained_word_embeddings/models/gold_pretrained/model14.bin'
optimizer = ner.begin_training()
# Now that we have our model, we can load in the pretrained weights.
with open(path_to_pretrained_weights, "rb") as file_:
    ner.model.tok2vec.from_bytes(file_.read())

In [ ]:
europe_pmc_sci_md = train_spacy(train_data,nlp,300,optimizer)

In [ ]:
# Save model 
from pathlib import Path

output_dir = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/iter_300/'
new_model_name = 'en_europepmc_sci_md'

if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.meta['name'] = new_model_name  # rename model
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)



In [ ]:
# Test the saved model
test_text = 'For the first time, a multi-step selection protocol, consisting in both analytical and biochemical tests, has been proposed in order to identify mold candidates to be tested as autochthonous co-starter (together with already selected yeasts and lactic acid bacteria) for black table olive production. '


print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
doc2 = nlp2(test_text)
for ent in doc2.ents:
    print(ent.label_, ent.text)

In [ ]:
test_text = 'Isolates recovered from DRBC agar medium were sequenced firstly in ITS region and only fungal spp. isolates were further sequenced in beta-tubulin gene'
doc2 = nlp(test_text)
for ent in doc2.ents:
    print(ent.label_, ent.text)

In [ ]:
nlp = spacy.load(model)  # load existing spacy model
print("Loaded model '%s'" % model)



In [ ]:
doc2 = nlp(test_text)
for ent in doc2.ents:
    print(ent.label_, ent.text)

In [ ]:
from nltk.tokenize import wordpunct_tokenize, WordPunctTokenizer
from typing import List
import re

In [ ]:
# Get full sentences from the XML
root_path = '/mnt/droplet/nfs/gns/literature/machine-learning/'
article_text_path = root_path+'evaluation/time_complexity/articles/'
path_to_PMCids = root_path+'benchmarking/benchmark_sent/'

############## The below function will extract text from XML files.

# evaluation_epmc_lib.extract_sentences_from_sentencised_xml_to_text_files(article_text_path, path_to_PMCids)

all_files = sorted(glob.glob(article_text_path + '*.txt*'))
result_json_dump_path = root_path+'evaluation/time_complexity/en-pubmed-pmc-lg/'
offset =26